In [33]:
from os import listdir
from os.path import isfile, join
import pandas as pd

path_base_original_real="C:\dataset\original_dataset\\real"
path_base_original_fake="C:\dataset\original_dataset\\fake"
fullpath_fake=path_base_original_fake+"\\fake.csv"
path_base_modified="C:\dataset\processed_datatset"
fullpath_modified=path_base_modified+"\\total_dataset.csv"
path_modified_onlyreal=path_base_modified+"\\real_dataset.csv"
path_modified_onlyfake=path_base_modified+"\\fake_dataset.csv"

In [34]:
import re 
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from gensim.models import word2vec


In [35]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\NEERAJ
[nltk_data]     KUMAR\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [36]:
def cleaningbody(txt):
    
    review = txt.split()
    if len(review)<15:
        return None
    
    review = re.sub('[^a-zA-Z]', ' ', txt) # Cleans all except characters
    review = review.lower()
    review = review.split()
    review = [   lemmatizer.lemmatize(word) for word in review if not word in set(stopwords.words('english'))  if len(word)<=3  ]
#     if review.size()<
#     ps = PorterStemmer()
#     review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    return review
    

In [37]:
smallerset_real_merged=path_base_modified+"smallerset_real_merged.csv"
df_real_merged=pd.read_csv(smallerset_real_merged)
df_real_merged.shape

(1000, 1)

In [38]:
smallerset_fake_merged=path_base_modified+"smallerset_fake_merged.csv"
df_fake_merged=pd.read_csv(smallerset_fake_merged)
df_fake_merged.shape

(1000, 1)

In [39]:
df_real_merged.isnull().sum().sum()

220

In [40]:
df_fake_merged.isnull().sum().sum()

2

In [41]:
df_real_merged.fillna("a",inplace=True)
df_fake_merged.fillna("a",inplace=True)

In [42]:
df_real_merged.isnull().sum().sum()

0

In [43]:
df_fake_merged.isnull().sum().sum()

0

In [44]:
df_real_merged.columns

Index(['content'], dtype='object')

In [45]:
df_real_merged['content'] =df_real_merged['content'].apply(lambda x: cleaningbody(x))
df_real_merged.to_csv(smallerset_real_merged,index=False,encoding="utf8")

In [46]:
df_fake_merged['content'] =df_fake_merged['content'].apply(lambda x: cleaningbody(x))
df_fake_merged.to_csv(smallerset_fake_merged,index=False,encoding="utf8")

In [47]:
df_real_merged['label']=1
df_fake_merged['label']=0
df=pd.concat([df_real_merged,df_fake_merged])
df.sample(frac=1)


,content,label
340,lot pew one big way new ad le le one one may a...,1
631,top two tie get get try vox com tie see u ear ...,0
117,lap day one cnn get,0
891,None,0
749,cnn pm two day cnn cnn two new fox tv cnn new ...,0
...,...,...
714,mia rob bex zoe ian see see see go god one say...,1
89,fan new one end ran new ago got jos end ago da...,0
695,war met way two one th u see sat jaw yes old o...,1
680,dog one dog big go man dog wag man one mr two ...,1


In [48]:
y=df.label
df=df.drop('label',axis=1)
df.fillna("",inplace=True)

In [157]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest= train_test_split(df['content'],y,test_size=0.32,random_state=60)

In [158]:
from sklearn.feature_extraction.text import CountVectorizer

In [159]:
vectorizer =CountVectorizer(stop_words='english')
trainx=vectorizer.fit_transform(xtrain)
testx=vectorizer.transform(xtest)

In [160]:
trainx.shape

(1360, 1264)

In [161]:
testx.shape

(640, 1264)

In [162]:
from sklearn.naive_bayes import MultinomialNB
classifier =MultinomialNB(alpha=0.03)
classifier.fit(trainx,ytrain)
testres=classifier.predict(testx)
from sklearn import metrics
score =metrics.accuracy_score(ytest,testres)
score

0.56875